In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

# Imports 

In [84]:
### Classes
from src.DataLoader import DataLoader
from src.Tensor import *
from interface.dataLoader_UI import Load_datas_pred
from interface.dataLoader_UI import Datas_truncature

import numpy as np
import os
import pandas as pd
import sys
from sklearn.model_selection import train_test_split


from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report,recall_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import GridSearchCV
from scipy import stats

import pickle

In [ ]:
file_path = "\\wsl.localhost\Ubuntu\home\marie\code\marieyalap\Project\Traian_Project\Datas\Datas_for_pred\test\test\Traian_walk_fresh"

In [1]:
pwd

'/home/marie/code/CapnJackSparrow/09-IMU-based-fatigue-detection/notebooks'

In [11]:
csv_path = "../../../marieyalap/Project/Traian_Project/Datas/Datas_for_pred/test/test/Traian_walk_fresh"

In [12]:
data_pred = Load_datas_pred(csv_path)

/home/marie/code/CapnJackSparrow/09-IMU-based-fatigue-detection/notebooks/../interface/dataLoader_UI.py:5: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{csv_file_path}/Accelerometer.csv", header=None)
/home/marie/code/CapnJackSparrow/09-IMU-based-fatigue-detection/notebooks/../interface/dataLoader_UI.py:24: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['UNIX_time'] = pd.to_datetime(df['UNIX_time'], unit='s')


In [28]:
data_pred

,Seconds_elapsed,Z_acceleration,Y_acceleration,X_acceleration
UNIX_time,,,,
2024-05-26 01:32:35.625000,-3.848350,-0.051010,0.000545,0.032262
2024-05-26 01:32:35.656250,-3.818350,-0.032744,-0.040288,-0.002768
2024-05-26 01:32:35.687500,-3.788350,0.080884,-0.128668,-0.154711
2024-05-26 01:32:35.718750,-3.753350,-0.272538,0.050467,0.013127
2024-05-26 01:32:35.750000,-3.718350,-0.288374,0.045536,-0.088909
...,...,...,...,...
2024-05-26 02:12:19.812500,2380.336676,-0.579128,-0.243219,-1.272636
2024-05-26 02:12:19.843750,2380.371589,0.125345,-0.725636,-0.573263
2024-05-26 02:12:19.875000,2380.406504,-0.398860,-1.167349,0.024975


In [ ]:
### 1) changing the order of columns => X, Y,Z => same names as the one from Data Loader
### 2) Find the middle and the number of lines in the datasets
### 3) truncatenate the datas in order to have 90 sec

In [20]:
#  df = df[cols]

#  cols = cols[-1:] + cols[:-1]

cols = data_pred.columns.tolist()
cols

['Seconds_elapsed', 'Z_acceleration', 'Y_acceleration', 'X_acceleration']

In [35]:
### 1) changing the order of columns => X, Y,Z => same names as the one from Data Loader

datas_pred_ordered = data_pred[["X_acceleration","Y_acceleration","Z_acceleration"]]
datas_pred_ordered

,X_acceleration,Y_acceleration,Z_acceleration
UNIX_time,,,
2024-05-26 01:32:35.625000,0.032262,0.000545,-0.051010
2024-05-26 01:32:35.656250,-0.002768,-0.040288,-0.032744
2024-05-26 01:32:35.687500,-0.154711,-0.128668,0.080884
2024-05-26 01:32:35.718750,0.013127,0.050467,-0.272538
2024-05-26 01:32:35.750000,-0.088909,0.045536,-0.288374
...,...,...,...
2024-05-26 02:12:19.812500,-1.272636,-0.243219,-0.579128
2024-05-26 02:12:19.843750,-0.573263,-0.725636,0.125345
2024-05-26 02:12:19.875000,0.024975,-1.167349,-0.398860


In [39]:
### 2) Find the middle and the number of lines in the datasets
len_datas = datas_pred_ordered.shape[0]
len_datas

76299

In [48]:
time_laps = 90
nb_event_time_laps = 32*time_laps
nb_event_time_laps
middle_target_time = np.round(len_datas/2)
middle_target_time

38150.0

In [56]:
begin_index_truncated = int(middle_target_time - (nb_event_time_laps/2))
begin_index_truncated

36710

In [55]:
end_index_truncated = int(middle_target_time + (nb_event_time_laps/2))
end_index_truncated

39590

In [53]:
## verification :
end_index_truncated - begin_index_truncated == nb_event_time_laps

True

In [47]:
nb_event_time_laps

2880

In [ ]:
### 3) truncatenate the datas in order to have 90 sec

In [59]:
df_pred_truncated = datas_pred_ordered.iloc[begin_index_truncated:end_index_truncated,:]
df_pred_truncated

,X_acceleration,Y_acceleration,Z_acceleration
UNIX_time,,,
2024-05-26 01:51:42.812500,0.180846,-1.581176,-0.194265
2024-05-26 01:51:42.843750,0.352007,-2.538964,-0.464634
2024-05-26 01:51:42.875000,0.453938,-3.033417,-0.205182
2024-05-26 01:51:42.906250,0.449854,-3.368692,0.028456
2024-05-26 01:51:42.937500,0.352493,-3.132666,0.295715
...,...,...,...
2024-05-26 01:53:12.656250,0.105730,-0.115864,1.141715
2024-05-26 01:53:12.687500,0.181555,3.696345,1.003456
2024-05-26 01:53:12.718750,-0.664856,6.125153,1.758174


In [63]:
## Test the function :
data_pred = Datas_truncature(Load_datas_pred(csv_path),90)
data_pred

/home/marie/code/CapnJackSparrow/09-IMU-based-fatigue-detection/notebooks/../interface/dataLoader_UI.py:6: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  # Remove the first row (index 0)
/home/marie/code/CapnJackSparrow/09-IMU-based-fatigue-detection/notebooks/../interface/dataLoader_UI.py:25: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  # Set the unix time as dataframe index


,X_acceleration,Y_acceleration,Z_acceleration
UNIX_time,,,
2024-05-26 01:51:42.812500,0.180846,-1.581176,-0.194265
2024-05-26 01:51:42.843750,0.352007,-2.538964,-0.464634
2024-05-26 01:51:42.875000,0.453938,-3.033417,-0.205182
2024-05-26 01:51:42.906250,0.449854,-3.368692,0.028456
2024-05-26 01:51:42.937500,0.352493,-3.132666,0.295715
...,...,...,...
2024-05-26 01:53:12.656250,0.105730,-0.115864,1.141715
2024-05-26 01:53:12.687500,0.181555,3.696345,1.003456
2024-05-26 01:53:12.718750,-0.664856,6.125153,1.758174


# NJ's code Try 

In [65]:
datas_m = []
for i in range(1,24):
    subject_ = f's{i}'
    data = DataLoader().load_ACC_data(subject=subject_,event='morning')
    data.drop(columns='UNIX_time',inplace=True)

    #new bit, NJ Monday
    data = data.reset_index(drop=True)
    #divides each df into 90 second chunks
    for chunk in range(20):
        #adds first 90 second chunk to the output list
        chunk_of_90 = data[:2880]
        flat_chunk_of_90 = chunk_of_90.to_numpy().flatten()
        datas_m.append(flat_chunk_of_90)
        #discards first 90 second chunk
        data = data.iloc[2880:]

In [66]:
datas_e = []
for i in range(1,24):
    subject_ = f's{i}'
    data = DataLoader().load_ACC_data(subject=subject_,event='evening')
    data.drop(columns='UNIX_time',inplace=True)

    #new bit, NJ Monday
    data = data.reset_index(drop=True)
    #divides each df into 90 second chunks
    for chunk in range(20):
        #adds first 90 second chunk to the output list
        chunk_of_90 = data[:2880]
        flat_chunk_of_90 = chunk_of_90.to_numpy().flatten()
        datas_e.append(flat_chunk_of_90)
        #discards first 90 second chunk
        data = data.iloc[2880:]

In [67]:
array = np.concatenate((datas_m, datas_e), axis=0)

In [68]:
def normalization(subject):
    num_transformer=make_pipeline(StandardScaler())
    num_transformer.fit(subject)
    return pd.DataFrame(num_transformer.transform(subject),columns=subject.columns)

In [69]:
datas = pd.DataFrame(array)

In [70]:
df=normalization(datas)

In [71]:
X = df

In [72]:
sleepy_list = [11, 18, 0, 12, 10, 11, 5, 11, 2, 5, 9, 9, 17, 4, 2, 12, 12, 19, 13, 7, 11, 3, 5, 14, 14, 14, 12, 13, 11, 1, 11, 13, 11, 8, 19, 21, 22, 16, 15, 12, 26, 4, 19, 21, 18, 20]

sleepy_df = pd.DataFrame(sleepy_list)

In [73]:
sleepy_df["fatigue"] = sleepy_df[0]

sleepy_df = sleepy_df.drop(columns=0)

sleepy_df["is_tired"] = 0

sleepy_df['is_tired'] = (sleepy_df['fatigue'] > 11).astype(int)

In [74]:
sleepy_df_20split = pd.DataFrame(np.repeat(sleepy_df.values, 20, axis=0))
sleepy_df_20split.columns = sleepy_df.columns
sleepy_df_20split.shape

(920, 2)

In [75]:
y = sleepy_df_20split.is_tired

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 128)

In [81]:
def test_classifiers(X_train, y_train, X_test, y_test):
    models = {
        'MLP Classifier': MLPClassifier(learning_rate="invscaling", hidden_layer_sizes=[4,4]),  ## --> This is your baseline = chance
        'Decision Tree': DecisionTreeClassifier(),
        #'Logistic Regression': LogisticRegression(),
        'Random Forest': RandomForestClassifier(),
        'Gradient Boosting': GradientBoostingClassifier(),
        'K-Nearest Neighbors': KNeighborsClassifier()

    }

    results = {}

    for name, model in models.items():
        pipe = make_pipeline(model)
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)

        report = classification_report(y_test, y_pred)

        results[name] = {
            'accuracy': accuracy,
            'report': report
        }

        print(f"{name}:\n")
        print(f"Accuracy: {accuracy}\n")
        print(f"Classification Report:\n{report}\n")

    return results

In [82]:
test_classifiers(X_train, y_train, X_test, y_test)

MLP Classifier:

Accuracy: 0.5652173913043478

Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.45      0.48       125
           1       0.59      0.66      0.62       151

    accuracy                           0.57       276
   macro avg       0.56      0.56      0.55       276
weighted avg       0.56      0.57      0.56       276


Decision Tree:

Accuracy: 0.5543478260869565

Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.57      0.54       125
           1       0.60      0.54      0.57       151

    accuracy                           0.55       276
   macro avg       0.56      0.56      0.55       276
weighted avg       0.56      0.55      0.56       276


Random Forest:

Accuracy: 0.6123188405797102

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.58      0.57       125
           1       0.65    

{'MLP Classifier': {'accuracy': 0.5652173913043478,
  'report': '              precision    recall  f1-score   support\n\n           0       0.52      0.45      0.48       125\n           1       0.59      0.66      0.62       151\n\n    accuracy                           0.57       276\n   macro avg       0.56      0.56      0.55       276\nweighted avg       0.56      0.57      0.56       276\n'},
 'Decision Tree': {'accuracy': 0.5543478260869565,
  'report': '              precision    recall  f1-score   support\n\n           0       0.51      0.57      0.54       125\n           1       0.60      0.54      0.57       151\n\n    accuracy                           0.55       276\n   macro avg       0.56      0.56      0.55       276\nweighted avg       0.56      0.55      0.56       276\n'},
 'Random Forest': {'accuracy': 0.6123188405797102,
  'report': '              precision    recall  f1-score   support\n\n           0       0.57      0.58      0.57       125\n           1       

# By model 

## MLP model 

In [116]:
model_mlp = MLPClassifier(shuffle=True)

In [118]:
param_grid = {
    'learning_rate': ['invscaling', 'adaptive'],
    #'subsample': [0.8, 0.9, 1.0]
    'hidden_layer_sizes' :[[3,3,3], [4,4], [5,5], [10,10], [20,10,5], [40,20,10]]
}

In [119]:
grid_search = GridSearchCV(estimator=model_mlp,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=2
             )

In [120]:
search = grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[CV] END hidden_layer_sizes=[3, 3, 3], learning_rate=invscaling; total time=   3.6s
[CV] END ..hidden_layer_sizes=[4, 4], learning_rate=adaptive; total time=   4.5s
[CV] END hidden_layer_sizes=[5, 5], learning_rate=invscaling; total time=   5.0s
[CV] END hidden_layer_sizes=[4, 4], learning_rate=invscaling; total time=   5.9s
[CV] END hidden_layer_sizes=[3, 3, 3], learning_rate=adaptive; total time=   6.9s
[CV] END hidden_layer_sizes=[3, 3, 3], learning_rate=adaptive; total time=   7.1s
[CV] END hidden_layer_sizes=[4, 4], learning_rate=invscaling; total time=   7.2s
[CV] END hidden_layer_sizes=[4, 4], learning_rate=invscaling; total time=   7.7s
[CV] END hidden_layer_sizes=[5, 5], learning_rate=invscaling; total time=   8.2s
[CV] END ..hidden_layer_sizes=[4, 4], learning_rate=adaptive; total time=   8.4s
[CV] END ..hidden_layer_sizes=[4, 4], learning_rate=adaptive; total time=   8.5s


/home/marie/.pyenv/versions/3.10.6/envs/walkies/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/marie/.pyenv/versions/3.10.6/envs/walkies/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END hidden_layer_sizes=[3, 3, 3], learning_rate=invscaling; total time=  10.1s
[CV] END hidden_layer_sizes=[3, 3, 3], learning_rate=invscaling; total time=  10.2s


/home/marie/.pyenv/versions/3.10.6/envs/walkies/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

/home/marie/.pyenv/versions/3.10.6/envs/walkies/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END hidden_layer_sizes=[20, 10, 5], learning_rate=adaptive; total time=   6.5s
[CV] END hidden_layer_sizes=[10, 10], learning_rate=invscaling; total time=  11.6s
[CV] END hidden_layer_sizes=[20, 10, 5], learning_rate=adaptive; total time=   7.9s
[CV] END hidden_layer_sizes=[20, 10, 5], learning_rate=invscaling; total time=  10.1s
[CV] END hidden_layer_sizes=[40, 20, 10], learning_rate=invscaling; total time=   8.0s
[CV] END hidden_layer_sizes=[40, 20, 10], learning_rate=adaptive; total time=   7.5s
[CV] END hidden_layer_sizes=[20, 10, 5], learning_rate=invscaling; total time=  11.6s
[CV] END hidden_layer_sizes=[20, 10, 5], learning_rate=adaptive; total time=  10.1s
[CV] END hidden_layer_sizes=[40, 20, 10], learning_rate=adaptive; total time=   7.1s
[CV] END hidden_layer_sizes=[40, 20, 10], learning_rate=invscaling; total time=  10.1s
[CV] END hidden_layer_sizes=[40, 20, 10], learning_rate=adaptive; total time=  10.9s


/home/marie/.pyenv/versions/3.10.6/envs/walkies/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END hidden_layer_sizes=[40, 20, 10], learning_rate=invscaling; total time=  14.5s


/home/marie/.pyenv/versions/3.10.6/envs/walkies/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [121]:
best_params = search.best_params_
best_params

{'hidden_layer_sizes': [20, 10, 5], 'learning_rate': 'adaptive'}

In [122]:
display(best_params['hidden_layer_sizes'])
display(best_params['learning_rate'])

[20, 10, 5]

'adaptive'

In [124]:
mlp_model_goodparams = MLPClassifier(learning_rate="adaptive", hidden_layer_sizes=[20, 10, 5])

In [125]:
mlp_model_goodparams.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=[20, 10, 5], learning_rate='adaptive')

In [126]:
y_pred = mlp_model_goodparams.predict(X_test)
y_pred

array([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0])

In [128]:
accuracy = accuracy_score(y_test, y_pred)

In [129]:
report = classification_report(y_test, y_pred)

In [130]:
print(f"Accuracy: {accuracy}\n")
print(f"Classification Report:\n{report}\n")

Accuracy: 0.5652173913043478

Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.55      0.53       125
           1       0.61      0.58      0.59       151

    accuracy                           0.57       276
   macro avg       0.56      0.56      0.56       276
weighted avg       0.57      0.57      0.57       276




## RandomForestClassifier

In [145]:
X_train.shape

(644, 8640)

In [146]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,8630,8631,8632,8633,8634,8635,8636,8637,8638,8639
252,-0.382259,0.690660,0.475532,-0.438804,0.700885,0.517267,-0.397119,0.698706,0.514123,-0.396111,...,-0.329169,-1.082272,-0.150668,-0.317578,-1.045216,-0.186775,-0.321602,-1.097893,-0.228934,-0.389174
68,1.212965,2.058302,0.409605,1.237049,2.069418,0.417730,1.298343,2.103669,0.447956,1.262029,...,0.847526,0.020884,0.197901,0.849553,0.016406,0.199800,0.850047,0.051262,0.198759,0.871871
765,0.255830,-0.129926,0.937018,0.195303,-0.120235,0.948593,0.244407,-0.120856,0.944207,0.202662,...,0.912898,0.066849,0.120442,0.914393,0.016406,0.122485,0.882593,0.005296,0.082115,0.839537
821,-1.476127,-0.442530,-0.579295,-1.208790,-0.628548,-0.511280,-1.588524,-0.042802,-0.048295,-1.593657,...,-0.296483,-1.128237,-0.073208,-0.252737,-1.045216,-0.070802,-0.223965,-1.189826,-0.034528,-0.227501
544,-0.792460,-0.129926,0.244788,-0.801150,-0.081134,0.251835,-0.809528,-0.120856,0.249456,-0.672468,...,-0.459913,-1.082272,-0.344317,-0.479679,-1.414476,-0.225432,-0.647060,-0.959995,-0.228934,-0.583181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,0.164675,0.143602,0.904055,0.150010,0.153471,0.915414,0.106938,0.152332,0.911124,0.156602,...,0.880212,0.066849,0.197901,0.881973,0.062563,0.199800,0.882593,0.051262,0.198759,0.871871
338,1.349698,0.299905,1.200725,1.327636,0.309875,1.214024,1.344166,0.308439,1.208874,1.354148,...,1.305130,1.583688,0.004252,1.206176,1.631918,0.006513,1.208051,1.660080,0.004353,1.227551
485,0.210253,-2.044626,-0.579295,0.421770,-1.997081,-0.411743,0.565170,-1.994140,-0.445295,0.847494,...,0.128435,-0.576659,-0.963995,0.136306,-0.491326,-0.959924,0.068948,-0.500332,-0.928794,0.095844
83,-0.838038,1.433094,-1.403377,-0.574684,1.209197,-1.307574,-0.672058,1.049947,-1.338547,-0.626409,...,0.324550,-0.714553,0.314091,0.460509,-0.814429,0.393088,0.557135,-0.362434,0.393164,0.354520


In [138]:
model_rdmF = RandomForestClassifier()

In [139]:
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    #'subsample': [0.8, 0.9, 1.0]
    'n_estimators': [200,300,400,500]
}

In [140]:
grid_search = GridSearchCV(estimator=model_rdmF,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=2
             )

In [141]:
search = grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END ...................criterion=gini, n_estimators=200; total time=   4.0s
[CV] END ...................criterion=gini, n_estimators=200; total time=   4.1s
[CV] END ...................criterion=gini, n_estimators=200; total time=   4.3s
[CV] END ................criterion=entropy, n_estimators=200; total time=   4.6s
[CV] END ................criterion=entropy, n_estimators=200; total time=   4.6s
[CV] END ................criterion=entropy, n_estimators=200; total time=   4.6s
[CV] END ...................criterion=gini, n_estimators=300; total time=   5.6s
[CV] END ...................criterion=gini, n_estimators=300; total time=   6.0s
[CV] END ...................criterion=gini, n_estimators=300; total time=   6.4s
[CV] END ................criterion=entropy, n_estimators=300; total time=   7.1s
[CV] END ...................criterion=gini, n_estimators=400; total time=   7.9s
[CV] END ...................criterion=gini, n_es

In [142]:
best_params = search.best_params_
best_params

{'criterion': 'log_loss', 'n_estimators': 400}

In [144]:
rdmF_model_goodparams = RandomForestClassifier(n_estimators=400, criterion='log_loss')
rdmF_model_goodparams

RandomForestClassifier(criterion='log_loss', n_estimators=400)

In [ ]:
mlp_model_goodparams.fit(X_train, y_train)

## Vote models

In [ ]:
voting_clf = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier()),
        ('mlp', MLPClassifier(learning_rate="invscaling", hidden_layer_sizes=[3,3,3])),
        ('gb', GradientBoostingClassifier())]
)

voting_clf.voting = "soft"
voting_clf.fit(X_train, y_train)
voting_clf.score(X_test, y_test)

In [ ]:
# Save the model to a file called 'model.pkl'
with open('voting_clf_model.pkl', 'wb') as file:
     pickle.dump(model, file)

# UI 